In [23]:
# import dask.dataframe as dd
from pyspark.sql import SparkSession
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.sql.functions import broadcast
from pyspark.sql.types import StructType
from pyspark.sql.functions import unix_timestamp, from_unixtime, col, udf
from pyspark.sql.types import DateType
from datetime import datetime
from pyspark.sql.functions import col, row_number
from pyspark.sql.window import Window
# sc = SparkContext('local')
spark = SparkSession(sc)
import pandas as pd
# from dataprep.clean import validate_country
import pyspark.sql.functions as F
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [10]:
df1 = spark.read.format("csv").option("header", "true").load("/home/jovyan/city_temperature_sample.csv")
df2 = spark.read.format("csv").option("header", "true").load("/home/jovyan/worldcities_sample.csv")
# df1.count()
# df2.count()


#Type cast day month and year to int 
df1=df1.withColumn('day',df1['day'].cast("int").alias('day'))
df1=df1.withColumn('month',df1['month'].cast("int").alias('month'))
df1=df1.withColumn('year',df1['year'].cast("int").alias('year'))
df1=df1.withColumn('AvgTemperature',df1['AvgTemperature'].cast("float").alias('AvgTemperature'))
df2=df2.withColumn('lat',df2['lat'].cast("float").alias('lat'))
df2=df2.withColumn('lng',df2['lng'].cast("float").alias('lng'))

df1.createOrReplaceTempView("city_temperature")
df2.createOrReplaceTempView("worldcities")

In [11]:

city_temperature1=spark.sql("""
select Region, country as temp_country, State, City as city_temp,
case when day >31 or day <1 then null else day end as day1, day,case when month >12 or month <1 then null else month end as month1,
month,year, case when length(year) <4 or length(year) >4 then null else year end year1,
AvgTemperature from city_temperature """)

city_temperature1.createOrReplaceTempView("city_temperature1")

In [12]:

df3=spark.sql("""
select Region, temp_country, State,city_temp, CONCAT(Day1, '/',  Month1, '/',Year1) as date_temp,
day, month,year,
AvgTemperature  from city_temperature1 """)
broadcast_df=df3.join(broadcast(df2),df3.temp_country==df2.country).cache()

#remove null field date fields

# broadcast_df.na.drop(Seq("date_temp")).show(false)
broadcast_df=broadcast_df.where(broadcast_df.date_temp.isNotNull())


In [13]:
broadcast_df.show()

+------+------------+-----+---------+---------+---+-----+----+--------------+-------+----------+-------+--------+-------+----+----+----------+-------+----------+----------+
|Region|temp_country|State|city_temp|date_temp|day|month|year|AvgTemperature|   city|city_ascii|    lat|     lng|country|iso2|iso3|admin_name|capital|population|        id|
+------+------------+-----+---------+---------+---+-----+----+--------------+-------+----------+-------+--------+-------+----+----+----------+-------+----------+----------+
|Africa|       Japan| null|  Algiers| 1/1/1995|  1|    1|1995|          64.2|Tokyo12|     Tokyo|35.6897|139.6922|  Japan| JP1| JPN|     Tōkyō|primary|  37977000|1392685764|
|Africa|       Japan| null|  Algiers|26/3/1995| 26|    3|1995|          54.7|Tokyo12|     Tokyo|35.6897|139.6922|  Japan| JP1| JPN|     Tōkyō|primary|  37977000|1392685764|
|Africa|       Japan| null|  Algiers|27/3/1995| 27|    3|1995|          54.5|Tokyo12|     Tokyo|35.6897|139.6922|  Japan| JP1| JPN|    

In [14]:
from pyspark.sql.functions import unix_timestamp, from_unixtime
func =  udf (lambda x: datetime.strptime(x, '%d/%m/%Y'), DateType())

broadcast_df1 = broadcast_df.withColumn('date_new', func(col('date_temp')))
broadcast_df1.printSchema()
broadcast_df1.show()


root
 |-- Region: string (nullable = true)
 |-- temp_country: string (nullable = true)
 |-- State: string (nullable = true)
 |-- city_temp: string (nullable = true)
 |-- date_temp: string (nullable = true)
 |-- day: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- AvgTemperature: float (nullable = true)
 |-- city: string (nullable = true)
 |-- city_ascii: string (nullable = true)
 |-- lat: float (nullable = true)
 |-- lng: float (nullable = true)
 |-- country: string (nullable = true)
 |-- iso2: string (nullable = true)
 |-- iso3: string (nullable = true)
 |-- admin_name: string (nullable = true)
 |-- capital: string (nullable = true)
 |-- population: string (nullable = true)
 |-- id: string (nullable = true)
 |-- date_new: date (nullable = true)

+------+------------+-----+---------+---------+---+-----+----+--------------+-------+----------+-------+--------+-------+----+----+----------+-------+----------+----------+----------+
|

In [19]:
#highest temperature  based on countries
w3 = Window.partitionBy("country").orderBy(col("AvgTemperature").desc())
broadcast_df1.withColumn("row",row_number().over(w3)) \
  .filter(col("row") == 1).drop("row") \
  .show()


+------+------------+-----+---------+---------+---+-----+----+--------------+-------+----------+-------+--------+-------+----+----+----------+-------+----------+----------+----------+
|Region|temp_country|State|city_temp|date_temp|day|month|year|AvgTemperature|   city|city_ascii|    lat|     lng|country|iso2|iso3|admin_name|capital|population|        id|  date_new|
+------+------------+-----+---------+---------+---+-----+----+--------------+-------+----------+-------+--------+-------+----+----+----------+-------+----------+----------+----------+
|Africa|       Japan| null|  Algiers| 1/1/1995|  1|    1|1995|          64.2|Tokyo12|     Tokyo|35.6897|139.6922|  Japan| JP1| JPN|     Tōkyō|primary|  37977000|1392685764|1995-01-01|
+------+------------+-----+---------+---------+---+-----+----+--------------+-------+----------+-------+--------+-------+----+----+----------+-------+----------+----------+----------+



In [25]:
broadcast_df1.withColumn("row",row_number().over(w3)) \
  .withColumn("avg", avg(col("AvgTemperature")).over(w3)) \
  .withColumn("sum", sum(col("AvgTemperature")).over(w3)) \
  .withColumn("min", min(col("AvgTemperature")).over(w3)) \
  .withColumn("max", max(col("AvgTemperature")).over(w3)) \
  .where(col("row")==1).select("country","avg","sum","min","max") \
  .show()

+-------+-----------------+------------------+----+----+
|country|              avg|               sum| min| max|
+-------+-----------------+------------------+----+----+
|  Japan|64.19999694824219|128.39999389648438|64.2|64.2|
+-------+-----------------+------------------+----+----+

